In [49]:
# What things can you do on your turn in Hanabi?
# 1. Play a card from your hand. -- Choose to play, choose a card.
# 2. Discard a card from your hand. -- Choose to discard, choose a card.
# 3. Give a hint to the other player about cards in their hand. -- Choose to give hint, choose a hint.

# What information do you have at any point in the Hanabi Game?
# 1. Hints given for each of your cards.
# 2. Number of remaining hints.
# 3. Number of cards left in the deck.
# 4. Number of lives left.
# 5. Hints given to other players already.
# 6. Cards in play.
# 7. Score.

# Maybe we can describe the state by [[score], [lives remaining], [hints available], [# cards in deck],
#                                     [Cards in play], [Hints in Hand], [Other People's Hands Hints]]

In [78]:
# We will have a game manager script that takes in a list of players and sends them the game state when it is their turn.
# We then instantiate a couple of players and have them play games trying to achieve the highest score.

In [202]:
import itertools
import random

class HanabiGame:
    
    def __init__(self, hand_size = 5, hints = 8, lives = 3, num_players = 2, num_suits = 5, num_values = 5):
                    
        # Hand Size
        if hand_size < 1:
            print("How are you going to play without any cards in your hand, silly?")
        self.hand_size = hand_size
        
        # Hints remaining
        self.hints = hints
        self.max_hints = self.hints
        
        # Lives remaining
        self.lives = lives

        # Number of players
        if num_players < 2:
            print("Hanabi must be played with at least two players.")
            
        self.num_players = num_players
        
        # Suits of Cards
        self.num_suits = num_suits
        self.suits = list(range(self.num_suits))

        # Values of Cards
        # There are 3 zeros of each suit, one of the maximal value, and two of all others.
        if num_values < 2:
            print("Please allow for at least two values.")
        
        self.num_values = num_values
        self.values = [0, 0, 0] + list(range(1, self.num_values-1))*2 + [self.num_values-1]

        # Deck
        # A card in the deck's ten's place is the suit of the card and 
        # the one's place is the value
        self.deck = [(suit,value) for suit, value in itertools.product(self.suits,self.values)]

        # Cards in the Hands of each Player
        self.player_hands = [[] for i in range(num_players)]
        self.initialDeal()

        # Hint information available to each player
        # self.player_hints[0] = [Hints received for suits, Hints received for values]
        # Hints received are either 0 (no info), 1 (positive info), or -1 (negative info)
        # So, [[-1, 0, 0, 0, 0], [-1, 1, -1, -1, -1]] would correspond to knowing a card is
        # NOT of suit 0 and IS of value 1.
        self.player_hints = [[[[0]*self.num_suits,[0]*self.num_values] for j in range(self.hand_size)]
                             for i in range(self.num_players)]
        
        # Keep track of whose turn it is
        self.current_player_turn = 0
        
        # Keep track of the score.
        # Note*: Normally in Hanabi your score is zero until the last turn
        # is played, but to help the AI, the current score will only become
        # zero when all lives are lost.
        self.current_score = 0
        
        # Cards that have been played onto the board
        # -1 corresponds to no card of a particular suit having been played.
        # The current value of self.board[i] = Highest value of card played
        # of that suit.
        self.board = [-1]*self.num_suits
        
        # Discard Pile
        # self.discards[i][j] = The number of suit i value j cards that have been discarded so far.
        self.discards = [[0]*num_values for i in range(self.num_suits)]
        
        # Keep a list of possible next moves
        self.legal_move_list = []
        # Initialize legal_move_list
        self.legalMoves()
        
        # Keep track of the number of moves left when the final round starts
        self.final_round_moves = self.num_players
        self.game_is_ending = False


    def initialDeal(self):
        random.shuffle(self.deck)
        for i in range(self.hand_size):
            for hand in range(self.num_players):
                self.player_hands[hand].append(self.deck.pop())
    
    # Returns the board state from a given player's perspective.
    # In particular, it returns all public information and the cards in the opponents hands.
    # [[score]: 1, [lives remaining]: 1, [hints available]: 1, [# cards in deck]: 1,
    # [Cards in play]: num_suits, [Discards]: num_suits*num_values, [Hints in Hand], [Other People's Hands Hints]]
    def boardState(self, player_number):
        return [self.current_score, self.lives, self.hints, len(self.deck), self.board, self.discards, 
                self.player_hints[player_number]] + [hint for i,hint in enumerate(self.player_hints) if i!= player_number]
    
    def printBoardState(self, player_number=-1):
        
        # If no player is specified, return the view from the current player
        if player_number == -1:
            player_number = self.current_player_turn
#         cur_board = boardState(player_number)
        print(f"From the perspective of player {player_number}")
        print(f"The current score is {self.current_score}.")
        print(f"You have {self.lives} lives and {self.hints} hints left.")
        print(f"There are {len(self.deck)} cards left in the deck.")
        for i, max_card in enumerate(self.board):
            if max_card == -1:
                print(f"You have not played any cards of suit {i}")
            elif max_card == 0:
                print(f"You have played the 0 of suit {i}.")
            else:
                print(f"You have played {max_card} cards of suit {i}.")
        
        for card_index, hint in enumerate(self.player_hints[player_number]):
            if hint == [[0]*self.num_suits, [0]*self.num_values]:
                print(f"You don't know anything about card {card_index}.")
            else:
                if hint[0] == [0]*self.num_suits:
                    print(f"You don't know anything about the suit of card {card_index}.")
                else:
                    for suit_index, existence in enumerate(hint[0]):
                        if existence != 0:
                            print(f"You know card {card_index} is {'not ' if existence == -1 else ''}of suit {suit_index}.")
                
                if hint[1] == [0]*self.num_values:
                    print(f"You don't know anything about the value of card {card_index}.")
                else:
                    for value_index, existence in enumerate(hint[1]):
                        if existence != 0:
                            print(f"You know card {card_index} is {'not ' if existence == -1 else ''}of value {value_index}.")

                            
    # Returns a tuple (The index of the current player's turn, [List of allowable moves])    
    def legalMoves(self):
        
        # Moves will be described by a tuple 
        # (Choose to Play/Discard/Hint, Index of Card to Discard/Play [-1 if giving hint instead], 
        #                              Hint Type[(player, suit, value)])
        # Hint Type is (-1,-1,-1) if not giving a hint.
        allowed_moves = []
        
        # Play moves (0,X,-1,-1)
        for i in range(len(self.player_hands[self.current_player_turn])):
            allowed_moves.append((0,i,-1,-1,-1))
        
        # Discard moves (1,X,-1,-1)
        for i in range(len(self.player_hands[self.current_player_turn])):
            allowed_moves.append((1,i,-1,-1,-1))
        
        
        # If hint available, hint moves (2, -1, Player, Suit, Value)
        if self.hints > 0:
            for player_index in range(self.num_players):
                if player_index != self.current_player_turn:
                    for suit_index in range(self.num_suits):
                        allowed_moves.append((2,-1,player_index,suit_index,-1))
                    for value_index in range(self.num_values):
                        allowed_moves.append((2,-1,player_index,-1,value_index))
        
        self.legal_move_list = allowed_moves
        return (self.current_player_turn, allowed_moves)
    
    def printLegalMoves(self):
        self.legalMoves()
        print(f"It is Player {self.current_player_turn}'s turn.")
        if len(self.legal_move_list) == 0:
            print("You done goofed. No legal moves.")
            return
        for index, move in enumerate(self.legal_move_list):
            if move[0] == 0:
                print(f"{index}: You can play your {move[1]} card.")
            elif move[0] == 1:
                print(f"{index}: You can discard your {move[1]} card.")
            else:
                if move[3] != -1:
                    print(f"{index}: You can tell player {move[2]} about their cards of suit {move[3]}.")
                elif move[4] != -1:
                    print(f"{index}: You can tell player {move[2]} about their cards of value {move[4]}.")
                else:
                    print('We definitely should not have gotten here.')
    
    # Current Player Draws a card
    def drawCard(self):
        if len(self.deck) == 0:
            self.game_is_ending = True
            self.final_round_moves -= 1
            if self.final_round_moves == 0:
                self.endGame()

        if len(self.player_hands[self.current_player_turn]) != 4:
            print(f"A player is trying to draw with {len(self.player_hands[self.current_player_turn])} cards in their hand.")
            return
        else:
            self.player_hands[self.current_player_turn].append(self.deck.pop())
            self.player_hints[self.current_player_turn].append([[0]*self.num_suits,[0]*self.num_values])
        
        self.legalMoves()
    
    # Implement this
    def endGame(self):
        return
    
    def playCard(self, index):
        if index > len(self.player_hands[self.current_player_turn]):
            print("That card is unplayable. Uh oh.")
            return
        
        card = self.player_hands[self.current_player_turn][index]
        suit = card[0]
        value = card[1]
        
        
        # If the card is playable, play it, increment score, and draw a card.
        # Otherwise, we'll lose a life, check game end, discard it, and draw a card.
        if self.board[suit] + 1 == value:
            self.board[suit] = value
            self.current_score += 1
        else:
            self.lives -= 1
            if self.lives <= 0:
                endGame()
                return
        self.discardCard(index, gain_hint = False)
        self.drawCard()

    # Only draw a card if gain_hint = True
    def discardCard(self, index, gain_hint = True):
        del self.player_hands[self.current_player_turn][index]
        del self.player_hints[self.current_player_turn][index]
        if gain_hint == True:
            self.drawCard()
            if self.hints < self.max_hints:
                self.hints += 1
        return

    # If a hint is given whilst game is ending, remember to decrement final_round_moves
    def giveHint(self, player, suit, value):
        return

    
    def performMove(self, index = -1, move = None):
        if index == -1 and move == None:
            print("What move would you like to do?")
            return
        if index > len(self.legal_move_list):
            print("Choose a valid move. [Index out of range.]")
            return
        if move != None and move not in legal_move_list:
            print("Choose a valid move. [Move not legal.]")
            return

        if index != -1:
            move = self.legal_move_list[index]
#         else:
#             index = legal_move_list.index(move)
        
        # Reminder: move = (play/discard/hint, card to play/discard index, player to give hint, suit hint, value hint)
        # Reminder: move[0]: 0 - Play, 1 - Discard, 2 - Hint
        if move[0] == 0:
            self.playCard(move[1])
        elif move[0] == 1:
            self.discardCard(move[1])
        elif move[0] == 2:
            self.giveHint((move[2], move[3], move[4]))
        
        # Next player's turn
        self.legalMoves()
        self.current_player_turn = (self.current_player_turn + 1) % self.num_players
        

In [203]:
test_game = HanabiGame()

In [204]:
test_game.printBoardState()

From the perspective of player 0
The current score is 0.
You have 3 lives and 8 hints left.
There are 40 cards left in the deck.
You have not played any cards of suit 0
You have not played any cards of suit 1
You have not played any cards of suit 2
You have not played any cards of suit 3
You have not played any cards of suit 4
You don't know anything about card 0.
You don't know anything about card 1.
You don't know anything about card 2.
You don't know anything about card 3.
You don't know anything about card 4.


In [205]:
test_game.printLegalMoves()

It is Player 0's turn.
0: You can play your 0 card.
1: You can play your 1 card.
2: You can play your 2 card.
3: You can play your 3 card.
4: You can play your 4 card.
5: You can discard your 0 card.
6: You can discard your 1 card.
7: You can discard your 2 card.
8: You can discard your 3 card.
9: You can discard your 4 card.
10: You can tell player 1 about their cards of suit 0.
11: You can tell player 1 about their cards of suit 1.
12: You can tell player 1 about their cards of suit 2.
13: You can tell player 1 about their cards of suit 3.
14: You can tell player 1 about their cards of suit 4.
15: You can tell player 1 about their cards of value 0.
16: You can tell player 1 about their cards of value 1.
17: You can tell player 1 about their cards of value 2.
18: You can tell player 1 about their cards of value 3.
19: You can tell player 1 about their cards of value 4.


In [206]:
test_game.performMove(index = 0)

In [207]:
test_game.printLegalMoves()

It is Player 1's turn.
0: You can play your 0 card.
1: You can play your 1 card.
2: You can play your 2 card.
3: You can play your 3 card.
4: You can play your 4 card.
5: You can discard your 0 card.
6: You can discard your 1 card.
7: You can discard your 2 card.
8: You can discard your 3 card.
9: You can discard your 4 card.
10: You can tell player 0 about their cards of suit 0.
11: You can tell player 0 about their cards of suit 1.
12: You can tell player 0 about their cards of suit 2.
13: You can tell player 0 about their cards of suit 3.
14: You can tell player 0 about their cards of suit 4.
15: You can tell player 0 about their cards of value 0.
16: You can tell player 0 about their cards of value 1.
17: You can tell player 0 about their cards of value 2.
18: You can tell player 0 about their cards of value 3.
19: You can tell player 0 about their cards of value 4.


In [208]:
test_game.printBoardState()

From the perspective of player 1
The current score is 1.
You have 3 lives and 8 hints left.
There are 39 cards left in the deck.
You have played the 0 of suit 0.
You have not played any cards of suit 1
You have not played any cards of suit 2
You have not played any cards of suit 3
You have not played any cards of suit 4
You don't know anything about card 0.
You don't know anything about card 1.
You don't know anything about card 2.
You don't know anything about card 3.
You don't know anything about card 4.


In [209]:
[i for i in range(10) if i!=2]

[0, 1, 3, 4, 5, 6, 7, 8, 9]

In [84]:
print(f"Is{' not' if True else ''}!")

Is not!
